In [1]:
import pandas as pd
from calendar import isleap

In [2]:
data_cap= pd.read_excel("Costo_Captacion_mensual.xlsx")

In [3]:
data_cap.set_index("Título", inplace= True)

In [5]:
data_udis= pd.read_excel("UDIS_Diario.xlsx")

In [6]:
data_udis.set_index("Fecha", inplace= True)

In [7]:
dc_dias= {"1": 31, "2":28, "3":31, "4": 30, "5":31, "6":30,
          "7": 31, "8": 31, "9":30, "10": 31, "11":30, "12":31}

In [8]:
aux=data_cap[(data_cap.index >= "2020-01-01") & (data_cap.index < "2022-01-20")].copy()

In [9]:
def get_days(aux, dia_i, dia_f):
    contador=0
    ls_dias= []
    size=len(aux) -1
    for index, row in aux.iterrows():
        if(contador==0):
            dias= dc_dias[str(index.month)]-dia_i +1
        elif(contador==size):
            dias= dia_f
        else:
            dias= dc_dias[str(index.month)]
        contador+=1
        if(index.month==2 and isleap(index.year)):
            dias+=1
        ls_dias.append(dias)
    aux["Dias"]= ls_dias
    return aux

In [10]:
tabla=get_days(aux, 1,21)

In [21]:
def calculateInterests(aux,udis_SA):
    tabla["Tasa anual"]= tabla["CCP_UDIS"].map(lambda x: (x*1.25)/100)
    tabla["Tasa diaria"]= tabla["Tasa anual"].map(lambda x: x/365)
    ls_interes= []
    for _,row in tabla.iterrows():
        tasa_diaria= row["Tasa diaria"]
        dias= row["Dias"]
        ls_interes.append(udis_SA*dias*tasa_diaria)
    tabla["Intereses moratorios"]= ls_interes
    
    return tabla

In [23]:
tabla=calculateInterests(aux,600000)

In [24]:
tabla["Intereses moratorios"].sum()

69800.75342465754